In [ ]:
import pandas as pd
import json
from tqdm import tqdm
from collections import defaultdict 
from collections import Counter
from os import walk

from scipy import stats
import numpy as np
import math
import statistics
import os
from collections import defaultdict
import ast
import spacy
nlp = spacy.load("en_core_web_sm")
import re

from scipy import stats
import math
import statistics
from scipy.spatial import distance

In [ ]:
data_path = "./LLM_Scored/run_config/"
filenames = list(set(next(walk(data_path), (None,None,[]))[2]))

496


## Number of recipes and average token length

In [3]:
total_count = 0.
coutn_eng = 0.
count_long = 0.
avg_len = []
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country']
    base_ingredient_list = recipe_dict['Reference_Base']['AllIngredients']


    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for recipe in llm_recipe_ids:
            recipe_ = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Instructions']
            title = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Title']
            ingredients = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Ingredients']
            avg_len.append(len(recipe_.split()) + len(title.split()) + len(ingredients))

    varia_countries = list(recipe_dict['LLM_gen']['variation_novelty'].keys())
    for varia in varia_countries:
        llm_recipe_varia_ids = list(recipe_dict['LLM_gen']['variation_novelty'][varia].keys())
        for i in range(len(llm_recipe_varia_ids)):
            recipe_ = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]['Instructions']
            title = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]['Title']
            ingredients = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]['Ingredients']
            avg_len.append(len(recipe_.split())+ len(title.split()) + len(ingredients))

100%|██████████| 501/501 [00:14<00:00, 33.82it/s]


In [ ]:
print('Number of recipes : ', len(avg_len))
print('AVG Recipe LENGTH : ', sum(avg_len)/len(avg_len))

196116
AVG Recipe LENGTH :  274.0281160129719


## Measure the divergence in ingredients

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def clean_ingr(ingredient, authorized_pos=['NOUN']):

    doc = nlp(ingredient)
    token_list = [token.text for token in doc if token.pos_ in authorized_pos]
    return token_list

In [5]:
dict_region = {"Asia": ["china", "mongolia", "india", "japan", "thailand", "vietnam", "singapore", "malaysia", "south korea", "pakistan", "philippines", "indonesia", "taiwan", "bangladesh", "cambodia", "uzbekistan", "azerbaijan"], 
"North America": ["united states", "mexico", "canada"], "Europe":["italy", "spain", "france", "switzerland", "hungary", "greece", "norway", "portugal", "russia", "sweden", "ireland", "germany", "romania", "finland", "belgium", "denmark", "slovakia", "netherlands", "guernsey", "luxembourg", "bulgaria", "lithuania", "armenia", "poland", "austria", "albania", "serbia", "moldova", "czech republic", "united kingdom", "liechtenstein", "latvia", "croatia", "macedonia", "ukraine", "belarus", "estonia", "bosnia and herzegovina", "malta", "cyprus", "iceland", "georgia"],
"Oceania":["australia", "fiji", "haiti", "brunei", "micronesia", "samoa", "tonga", "guam"], "South America": ["peru", "brazil", "panama", "bolivia", "uruguay", "el salvador", "colombia", "argentina", "paraguay", "nicaragua", "venezuela", "guatemala", "guyana", "ecuador", "honduras"], 
"Carribean": ["cuba", "jamaica", "trinidad and tobago", "barbados", "bermuda", "belize", "antigua and barbuda", "bahamas", "dominican republic", "grenada", "curaçao"],
"East Asia": ["israel",  "palestinian territories", "turkiye", "lebanon", "afghanistan", "tunisia", "iraq", "iran", "qatar", "syria", "jordan", "oman", "bahrain", "yemen"],
"Africa": ["egypt", "morocco", "algeria", "ethiopia", "guinea", "mauritius", "madagascar", "chad", "mozambique", "senegal", "kenya", "libya", "cote d’ivoire", "ghana", "angola", "mauritania", "nigeria", "congo", "liberia"]}

country_to_region = {country: region for region, countries in dict_region.items() for country in countries}


We measure the proportion of new ingredients in these recipes that are also present in basic recipes, compared to humans as well.

In [7]:
prop_HiR, prop_RiH, prop_LiR, prop_RiL = [],[],[],[]
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country']
    base_ingredient_list = recipe_dict['Reference_Base']['AllIngredients']

    LLM_ingredient_list = []
    human_ingredient_list = []
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for recipe in llm_recipe_ids:
            ingredient_list = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Ingredients']
            for ingredient in ingredient_list:
                LLM_ingredient_list.extend(clean_ingr(ingredient))

    valid_recipe_ids = list(recipe_dict['Valid_Variations'].keys())
    for i in range(len(valid_recipe_ids)):
        if recipe_dict['Valid_Variations'][valid_recipe_ids[i]]['country'] == country_orig:
            ingredient_valid = recipe_dict['Valid_Variations'][valid_recipe_ids[i]]['ingredient_list']
            for ingredient in ingredient_valid:
                human_ingredient_list.extend(clean_ingr(ingredient))
    
    test_recipe_ids = list(recipe_dict['Test_Variations'].keys())
    for i in range(len(test_recipe_ids)):
        if recipe_dict['Test_Variations'][test_recipe_ids[i]]['country'] == country_orig:
            ingredient_test = recipe_dict['Test_Variations'][test_recipe_ids[i]]['ingredient_list']
            for ingredient in ingredient_valid:
                human_ingredient_list.extend(clean_ingr(ingredient))

    if len(human_ingredient_list) >0:
        human_ingredient_list = list(set(human_ingredient_list))
        human_in_ref = sum(1 for item in human_ingredient_list if item in base_ingredient_list)
        prop_HiR.append(human_in_ref / len(human_ingredient_list))
        ref_in_human = sum(1 for item in base_ingredient_list if item in human_ingredient_list)
        prop_RiH.append(ref_in_human / len(base_ingredient_list))
    else:
        prop_HiR.append(0.)
        prop_RiH.append(0.)
    if len(LLM_ingredient_list) >0:
        LLM_ingredient_list = list(set(LLM_ingredient_list))
        LLM_in_ref = sum(1 for item in LLM_ingredient_list if item in base_ingredient_list)
        prop_LiR.append(LLM_in_ref / len(LLM_ingredient_list))
        ref_in_LLM = sum(1 for item in base_ingredient_list if item in LLM_ingredient_list)
        prop_RiL.append(ref_in_LLM / len(base_ingredient_list))
    else:
        prop_LiR.append(0.)
        prop_RiL.append(0.)

100%|██████████| 501/501 [21:29<00:00,  2.57s/it]


In [ ]:
prop_HiR = np.array(prop_HiR)
prop_RiH = np.array(prop_RiH)
prop_LiR = np.array(prop_LiR)
prop_RiL = np.array(prop_RiL)

# Step 1: Mean values
mean_HiR, mean_RiH, mean_LiR, mean_RiL = np.mean(prop_HiR), np.mean(prop_RiH), np.mean(prop_LiR), np.mean(prop_RiL)
print("Mean Human in base:", 1 - mean_HiR, "Mean Base in Human:", 1-mean_RiH, "Mean LLM in base:", 1-mean_LiR, "Mean Base in LLM:", 1-mean_RiL)

Mean Human in base: 0.9959826655934441 Mean Base in Human: 0.9999189378295624 Mean LLM in base: 0.9334478646658051 Mean Base in LLM: 0.7904652996228437


In [6]:
## Doing it by region
prop_LiR, prop_RiL = {}, {}

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country']
    base_ingredient_list = recipe_dict['Reference_Base']['AllIngredients']

    if country_orig in country_to_region.keys():
        region = country_to_region.get(country_orig)
    else:
        region = 'UNK'

    LLM_ingredient_list = []
    human_ingredient_list = []
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for recipe in llm_recipe_ids:
            ingredient_list = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Ingredients']
            for ingredient in ingredient_list:
                LLM_ingredient_list.extend(clean_ingr(ingredient))


    if len(LLM_ingredient_list) > 0:
        LLM_ingredient_list = list(set(LLM_ingredient_list))
        LLM_in_ref = sum(1 for item in LLM_ingredient_list if item in base_ingredient_list)
        if region not in prop_LiR.keys():
            prop_LiR[region] = []
        
        prop_LiR[region].append(LLM_in_ref / len(LLM_ingredient_list))
        
        ref_in_LLM = sum(1 for item in base_ingredient_list if item in LLM_ingredient_list)
        if region not in prop_RiL.keys():
            prop_RiL[region] = []
        prop_RiL[region].append(ref_in_LLM / len(base_ingredient_list))

100%|██████████| 496/496 [04:02<00:00,  2.05it/s]


In [14]:
## Regions  Asia | Europe | North America | Oceania | South America | Carribean | East Asia | Africa
region = "Asia"
prop_LiR_Regional = np.array(prop_LiR[region])
prop_RiL_Regional = np.array(prop_RiL[region])

# Step 1: Mean values
mean_LiR, mean_RiL =  np.mean(prop_LiR_Regional), np.mean(prop_RiL_Regional)
print("Mean LLM in base:", 1-mean_LiR, "Mean Base in LLM:", 1-mean_RiL)

Mean LLM in base: 0.627067873664686 Mean Base in LLM: 0.8875215797678363


## Counting the number of time ingredients list is used in instructions

In [5]:
prop_ingr_in_instr = []
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)
    
    country_orig = recipe_dict['Country']
    base_ingredient_list = recipe_dict['Reference_Base']['AllIngredients']
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for recipe in llm_recipe_ids:
            count_in_instructions = 0
            ingredient_list = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Ingredients']
            individual_ingredient = []
            for ingredient in ingredient_list:
                individual_ingredient.extend(clean_ingr(ingredient))

            recipe = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Instructions']
            
            for ingredient in individual_ingredient:
                if ingredient in recipe:
                    count_in_instructions += 1
            
            if len(individual_ingredient) > 0:
                individual_ingredient = list(set(individual_ingredient))
                prop_ingr_in_instr.append(count_in_instructions / len(individual_ingredient))

100%|██████████| 501/501 [06:26<00:00,  1.30it/s]


In [6]:
prop_ingr_in_instr  = np.array(prop_ingr_in_instr)
mean_IiI = np.mean(prop_ingr_in_instr)
print("Mean Ingre in Instructions Same country:", mean_IiI)

Mean Ingre in Instructions Same country: 0.5901024763876807


In [7]:
prop_ingr_in_instr = []
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)
    
    country_orig = recipe_dict['Country']
    base_ingredient_list = recipe_dict['Reference_Base']['AllIngredients']


    varia_countries = list(recipe_dict['LLM_gen']['variation_novelty'].keys())
    for varia in varia_countries:
        llm_recipe_varia_ids = list(recipe_dict['LLM_gen']['variation_novelty'][varia].keys())
        for i in range(len(llm_recipe_varia_ids)):
            count_in_instructions = 0
            ingredient_list = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]['Ingredients']
            for ingredient in ingredient_list:
                individual_ingredient.extend(clean_ingr(ingredient))
                
            recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]['Instructions']

            for ingredient in individual_ingredient:
                if ingredient in recipe:
                    count_in_instructions += 1
            
            if len(individual_ingredient) > 0:
                individual_ingredient = list(set(individual_ingredient))
                prop_ingr_in_instr.append(count_in_instructions / len(individual_ingredient))

100%|██████████| 501/501 [1:08:22<00:00,  8.19s/it]


In [8]:
prop_ingr_in_instr  = np.array(prop_ingr_in_instr)
mean_IiI = np.mean(prop_ingr_in_instr)
print("Mean Ingre in Instructions Varia country:", mean_IiI)

Mean Ingre in Instructions Varia country: 0.019365754640101804


In [9]:
prop_2  = np.array(prop_ingr_in_instr[0:100])
mean_IiI = np.mean(prop_2)
print("Mean Ingre in Instructions Varia country:", mean_IiI)

Mean Ingre in Instructions Varia country: 0.3415972864592488


## Counting non-english and too short instructions

In [15]:
from langdetect import detect

In [16]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [17]:
def count_tokens(recette, thrs=50):
    list_recette = recette.split()
    if len(list_recette) > thrs:
        return True
    else:
        return False


In [18]:
total_count = 0.
coutn_eng = 0.
count_long = 0.
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country']
    base_ingredient_list = recipe_dict['Reference_Base']['AllIngredients']

    LLM_ingredient_list = []
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for recipe in llm_recipe_ids:
            recipe = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Instructions']
            if is_english(recipe):
                coutn_eng += 1
            if count_tokens(recipe):
                count_long +=1 
            total_count += 1
    
    varia_countries = list(recipe_dict['LLM_gen']['variation_novelty'].keys())
    for varia in varia_countries:
        llm_recipe_varia_ids = list(recipe_dict['LLM_gen']['variation_novelty'][varia].keys())
        for i in range(len(llm_recipe_varia_ids)):
            recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]['Instructions']
            if is_english(recipe):
                coutn_eng += 1
            if count_tokens(recipe):
                count_long +=1 
            total_count += 1

100%|██████████| 501/501 [21:48<00:00,  2.61s/it]


In [20]:
print('Nb tot recipes : ', total_count)
print('Prop_eng : ', coutn_eng/total_count)
print('Prop more than 50 tokens : ', count_long/total_count)

Nb tot recipes :  196116.0
Prop_eng :  0.9572038997328112
Prop more than 50 tokens :  0.9712568071957413


## Counting when country is explicitely not in the countru of origin or the variation asked

In [21]:
import re

In [22]:
country_df = pd.read_csv('/home/mila/f/florian.carichon/scratch/Cultural_Datasets/countries.csv')
lower_countries_df = country_df.assign(nat_lower=country_df['Nationality'].str.strip().str.lower())
lower_countries_df = lower_countries_df.assign(country_lower=country_df['Name'].str.strip().str.lower())
nat_to_country = lower_countries_df.drop_duplicates('nat_lower').set_index('nat_lower')['Name'].to_dict()

country_list = list(lower_countries_df['Name'])
country_list_lower = list(lower_countries_df['country_lower'])

nationalty_list = list(lower_countries_df['Nationality'])
nationalty_list_lower = list(lower_countries_df['nat_lower'])

In [23]:
dict_orig = {}
same_orig_tradi = 0
same_varia = 0
tot_count = 0

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country'].lower()
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        same_novelty_dict = recipe_dict['LLM_gen']['same_country_novelty']
        same_key_list = list(same_novelty_dict.keys())

        final_count_list = []
        for key in same_key_list:
            if "basic" in key.lower():          #those are the recipes where I don't mention the country of origin
                title = same_novelty_dict[key]['Title']
                tokens = re.findall(r'\b\w+\b', title.lower())
                matches_countries = [token for token in tokens if token in country_list_lower]
                matches_nat = [token for token in tokens if token in nationalty_list_lower]
                matches_countries = matches_countries + [nat_to_country.get(nat.strip().lower()) for nat in matches_nat]
                final_count_list.extend(matches_countries)
            else:
                title = same_novelty_dict[key]['Title']
                tokens = re.findall(r'\b\w+\b', title.lower())
                matches_countries = [token for token in tokens if token in country_list_lower]
                matches_nat = [token for token in tokens if token in nationalty_list_lower]
                matches_countries = matches_countries + [nat_to_country.get(nat.strip().lower()) for nat in matches_nat]
                if len(matches_countries) == 0:
                    same_varia += 1
                    tot_count += 1
                else:
                    for country in matches_countries:
                        if country == country_orig:
                            same_varia += 1
                        tot_count += 1

        if len(final_count_list) > 0:
            count_dict = dict(Counter(final_count_list))
            most_common_country = max(count_dict, key=count_dict.get)
            dict_orig[country_orig] = most_common_country.lower()
            if most_common_country.lower() == country_orig:
                same_orig_tradi += 1
        

    varia_novelty_dict = recipe_dict['LLM_gen']['variation_novelty']
    varia_country_keys = list(varia_novelty_dict.keys())

    for varia_country in varia_country_keys:
        country_v = varia_country.split("_", 1)[1]
        key_list = list(varia_novelty_dict[varia_country].keys())
        for key in key_list:
            title = varia_novelty_dict[varia_country][key]['Title']
            tokens = re.findall(r'\b\w+\b', title.lower())
            matches_countries = [token for token in tokens if token in country_list_lower]
            matches_nat = [token for token in tokens if token in nationalty_list_lower]
            matches_countries = matches_countries + [nat_to_country.get(nat.strip().lower()) for nat in matches_nat]
            if len(matches_countries) == 0:
                same_varia += 1
                tot_count += 1
            else:
                for country in matches_countries:
                    if country == country_orig:
                        same_varia += 1
                    tot_count += 1

100%|██████████| 501/501 [00:20<00:00, 24.30it/s]


In [24]:
print(same_varia, tot_count, same_varia/tot_count)
print(same_orig_tradi, 1- same_orig_tradi/496) ##501
print(dict_orig)

131848 203577 0.6476566606247267
179 0.6391129032258065
{'italy': 'italy', 'china': 'south korea', 'mexico': 'cuba', 'lebanon': 'morocco', 'russia': 'russia', 'germany': 'pennsylvania', 'cuba': 'cuba', 'india': 'india', 'netherlands': 'netherlands', 'greece': 'greece', 'morocco': 'south korea', 'ukraine': 'ukraine', 'poland': 'poland', 'thailand': 'india', 'portugal': 'portugal', 'united states': 'united states', 'france': 'france', 'philippines': 'philippines', 'south korea': 'morocco', 'spain': 'spain', 'tunisia': 'morocco', 'hungary': 'hungary', 'armenia': 'armenia', 'denmark': 'denmark', 'brazil': 'brazil', 'australia': 'australia', 'taiwan': 'china', 'syria': 'lebanon', 'malaysia': 'india', 'nigeria': 'morocco', 'jamaica': 'jamaica', 'ireland': 'morocco', 'serbia': 'poland', 'colombia': 'colombia', 'sweden': 'morocco', 'switzerland': 'italy', 'japan': 'japan', 'romania': 'romania', 'peru': 'argentina', 'vietnam': 'switzerland', 'argentina': 'argentina', 'algeria': 'morocco', 'mong

## Counting recipes with less than 50 tokens & repetition of tokens

In [25]:
too_short = 0
recipe_count = 0
window = 3
repeat_prop = []
country_repeat = []
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country'].lower()
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for recipe in llm_recipe_ids:
            recipe = recipe_dict['LLM_gen']['same_country_novelty'][recipe]['Instructions']
            tokens = recipe.split(' ')
            if len(tokens) <= 50:
                too_short += 1
            recipe_count += 1
            repeats = 0
            for i, token in enumerate(tokens):
                # look ahead up to 3 tokens
                if token in tokens[i+1:i+1+window]:
                    repeats += 1
            repeat_prop.append(repeats/len(tokens))
            if repeats/len(tokens) > 0.1:
                country_repeat.append(country_orig)

    
    
    varia_countries = list(recipe_dict['LLM_gen']['variation_novelty'].keys())
    for varia in varia_countries:
        llm_recipe_varia_ids = list(recipe_dict['LLM_gen']['variation_novelty'][varia].keys())
        for i in range(len(llm_recipe_varia_ids)):
            recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]['Instructions']
            tokens = recipe.split(' ')
            if len(tokens) <= 50:
                too_short += 1
            recipe_count += 1
            repeats = 0
            for i, token in enumerate(tokens):
                # look ahead up to 3 tokens
                if token in tokens[i+1:i+1+window]:
                    repeats += 1
            repeat_prop.append(repeats/len(tokens))
            if repeats/len(tokens) > 0.1:
                country_repeat.append(country_orig)

100%|██████████| 501/501 [00:22<00:00, 22.68it/s]


In [26]:
print(too_short/recipe_count)
print(sum(repeat_prop)/len(repeat_prop))
print(Counter(country_repeat))

0.028743192804258706
0.005526344479807427
Counter({'mexico': 13, 'italy': 11, 'france': 11, 'united states': 9, 'china': 7, 'thailand': 7, 'india': 7, 'switzerland': 7, 'greece': 6, 'germany': 5, 'morocco': 4, 'russia': 4, 'spain': 3, 'sweden': 3, 'brazil': 3, 'japan': 3, 'south korea': 3, 'turkiye': 3, 'ireland': 3, 'portugal': 2, 'netherlands': 2, 'philippines': 2, 'vietnam': 2, 'cuba': 2, 'ukraine': 1, 'australia': 1, 'taiwan': 1, 'malaysia': 1, 'tunisia': 1, 'algeria': 1, 'congo': 1, 'serbia': 1, 'ethiopia': 1})


## Counting Title where recipe name or country is not in the title 

In [27]:
recipe_presence = 0
ingredient_presence = 0
total = 0

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country'].lower()
    recipe_name = recipe_dict['Recipe_Name'].lower()
    recipe_name = recipe_name.split(' ')
    #print(recipe_name)
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        same_novelty_dict = recipe_dict['LLM_gen']['same_country_novelty']
        same_key_list = list(same_novelty_dict.keys())

        for key in same_key_list:
            recipe_prop = 0
            ingredient_prop = 0
            title = same_novelty_dict[key]['Title']
            ingredient_list = same_novelty_dict[key]['Ingredients']
            for noun in recipe_name:
                if noun in title:
                    recipe_prop += 1
                if any(noun in ingredient for ingredient in ingredient_list):
                    ingredient_prop += 1
            if recipe_prop / len(recipe_name) > 0.5:
                recipe_presence += 1
            if ingredient_prop / len(recipe_name) > 0.5:
                ingredient_presence += 1
            total += 1

    varia_novelty_dict = recipe_dict['LLM_gen']['variation_novelty']
    varia_country_keys = list(varia_novelty_dict.keys())

    for varia_country in varia_country_keys:
        country_v = varia_country.split("_", 1)[1]
        key_list = list(varia_novelty_dict[varia_country].keys())
        for key in key_list:
            title = varia_novelty_dict[varia_country][key]['Title']
            ingredient_list = varia_novelty_dict[varia_country][key]['Ingredients']
            for noun in recipe_name:
                if noun in title:
                    recipe_prop += 1
                if any(noun in ingredient for ingredient in ingredient_list):
                    ingredient_prop += 1
            if recipe_prop / len(recipe_name) > 0.5:
                recipe_presence += 1
            if ingredient_prop / len(recipe_name) > 0.5:
                ingredient_presence += 1
            total += 1

100%|██████████| 501/501 [00:12<00:00, 40.32it/s]


In [28]:
print('Proportion of recipe name that are in the title of the recipe explicitely', recipe_presence/total)
print('Proportion of recipe that are in the list of ingredients', ingredient_presence/total)

Proportion of recipe name that are in the title of the recipe explicitely 0.9181504823675783
Proportion of recipe that are in the list of ingredients 0.8672622325562422


## From which country the recipes are coming from?
Based on the common ingredients

In [15]:
dict_ingredients_country = defaultdict(list)

In [ ]:
## Creating the list of ingredients per country for humans -- Execute one only
ingredient_path = './country_ingredients.json'

In [17]:
if not os.path.exists(ingredient_path):
    print("File not found — generating it now...")
    
    for k in tqdm(range(len(filenames))):
        recette = filenames[k]
        file_path = data_path + recette
        with open(file_path) as json_file:
            recipe_dict = json.load(json_file)

        #Appending first the list of ingredients of the associated country of origin
        country_orig = recipe_dict['Country'].lower()
        dict_ingredients_country[country_orig] = recipe_dict['Reference_Base']['AllIngredients']

        #Going through training/var/test to append also list of ingredients
        id_train = recipe_dict['Train_Variations']
        for id in id_train:
            country_v = recipe_dict['Train_Variations'][id]['country'].lower()
            str_ingredients = recipe_dict['Train_Variations'][id]['ingredient_list']
            ingredient_list = ast.literal_eval(str_ingredients)
            if country_v not in dict_ingredients_country.keys():
                dict_ingredients_country[country_v] = ingredient_list
            else:
                dict_ingredients_country[country_v].extend(ingredient_list)

        id_valid = recipe_dict['Valid_Variations']
        for id in id_valid:
            country_v = recipe_dict['Valid_Variations'][id]['country'].lower()
            str_ingredients = recipe_dict['Valid_Variations'][id]['ingredient_list']
            ingredient_list = ast.literal_eval(str_ingredients)
            if country_v not in dict_ingredients_country.keys():
                dict_ingredients_country[country_v] = ingredient_list
            else:
                dict_ingredients_country[country_v].extend(ingredient_list)

        id_test = recipe_dict['Test_Variations']
        for id in id_test:
            country_v = recipe_dict['Test_Variations'][id]['country'].lower()
            str_ingredients = recipe_dict['Test_Variations'][id]['ingredient_list']
            ingredient_list = ast.literal_eval(str_ingredients)
            if country_v not in dict_ingredients_country.keys():
                dict_ingredients_country[country_v] = ingredient_list
            else:
                dict_ingredients_country[country_v].extend(ingredient_list)
        
        with open(ingredient_path, "w") as f:
            json.dump(dict_ingredients_country, f, indent=2)
else:
    print("File already exists — loading it.")
    with open(ingredient_path, "r") as f:
        dict_ingredients_country = json.load(f)

File already exists — loading it.


In [ ]:
def find_best_match(list_orig, dict_of_lists):

    best_key = None
    best_count = 0

    set_orig = set(list_orig)  # use set for fast intersection

    for key, values in dict_of_lists.items():
        common = set_orig.intersection(set(values))
        count = len(common)

        if count > best_count:
            best_count = count
            best_key = key

    return best_key, best_count

In [19]:

def clean_ingredients_spacy(lines, list_countries):
    """
    Keep only nouns (and noun→noun compounds), remove countries.
    lines: list of raw ingredient strings (with leading "- " etc.)
    list_countries: list of country names to filter out (any case)
    """
    # normalize country names to lowercase lemmas for filtering
    countries = {c.strip().lower() for c in list_countries}

    cleaned = []
    for raw in lines:
        # 1) remove leading "- " and lowercase for uniformity
        s = re.sub(r'^\s*-\s*', '', raw).strip()

        doc = nlp(s)
        phrases = []
        for tok in doc:
            # head noun or proper noun
            if tok.pos_ in {"NOUN", "PROPN"} and tok.dep_ not in {"compound"}:
                # build phrase: include left-side noun compounds (NOUN/PROPN with dep_ == 'compound')
                parts = []
                for left in tok.lefts:
                    if left.dep_ == "compound" and left.pos_ in {"NOUN", "PROPN"}:
                        parts.append(left.lemma_.lower())
                parts.append(tok.lemma_.lower())

                # drop any country tokens inside the phrase
                parts = [p for p in parts if p not in countries]
                if parts:
                    phrases.append(" ".join(parts))

        # 4) simple heuristic:
        #    if we found multiple noun phrases, keep the longest one,
        #    else keep the first, else empty string
        if phrases:
            best = max(phrases, key=lambda x: len(x))
            cleaned.append(best)
        else:
            cleaned.append("")

    return cleaned

In [17]:
dict_comparision = defaultdict(list)
avg_best_match_ingre = []
same_country = 0
total = 0
no_match = 0 
total_m = 0
list_countries_filter = list(dict_ingredients_country.keys())

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country'].lower()
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        same_novelty_dict = recipe_dict['LLM_gen']['same_country_novelty']
        same_key_list = list(same_novelty_dict.keys())
        for key in same_key_list:
            ingredient_list = same_novelty_dict[key]['Ingredients']
            ingredient_list = clean_ingredients_spacy(ingredient_list, list_countries_filter)
            best_country, best_count = find_best_match(ingredient_list, dict_ingredients_country)
            if best_country != None:
                avg_best_match_ingre.append(best_count)
                if country_orig not in dict_comparision.keys():
                    dict_comparision[country_orig] = [best_country]
                else:
                    dict_comparision[country_orig].append(best_country)

                if best_country == country_orig:
                    same_country +=1
                total_m += 1
                total += 1
            else:
                no_match += 1
                total += 1

    
    varia_novelty_dict = recipe_dict['LLM_gen']['variation_novelty']
    varia_country_keys = list(varia_novelty_dict.keys())

    for varia_country in varia_country_keys:
        country_v = varia_country.split("_", 1)[1]
        key_list = list(varia_novelty_dict[varia_country].keys())
        for key in key_list:
            title = varia_novelty_dict[varia_country][key]['Title']
            ingredient_list = varia_novelty_dict[varia_country][key]['Ingredients']
            ingredient_list = clean_ingredients_spacy(ingredient_list, list_countries_filter)
            best_country, best_count = find_best_match(ingredient_list, dict_ingredients_country)
            if best_country != None:
                avg_best_match_ingre.append(best_count)
                if country_v not in dict_comparision.keys():
                    dict_comparision[country_v] = [best_country]
                else:
                    dict_comparision[country_v].append(best_country)
                
                if best_country == country_v:
                    same_country +=1
                total+=1
            else:
                no_match += 1
                total += 1

100%|██████████| 484/484 [00:45<00:00, 10.60it/s]


In [19]:
print('Proportion of recipes where ingredients match their expected country : ', same_country/total)
print('Proportion of absolutely not match ingredients -- the recipes used 0 ever seen ingredients : ', no_match/total)
all_items = [item for sublist in dict_comparision.values() for item in sublist]
counts = Counter(all_items)
#most_common_item, frequency = counts.most_common(1)[0]
top5 = counts.most_common(10)
print(f"Most common countries:", top5)

Proportion of recipes where ingredients match their expected country :  0.005191833861316438
Proportion of absolutely not match ingredients -- the recipes used 0 ever seen ingredients :  0.944385779655051
Most common countries: [('china', 316), ('chile', 198), ('united states', 71), ('australia', 69), ('mexico', 67), ('mongolia', 61), ('russia', 58), ('india', 54), ('hungary', 52), ('sweden', 37)]


# Inrgedients from Cosine Similiarity

In [ ]:
ingredient_path = './country_ingredients.json'
with open(ingredient_path, "r") as f:
    dict_ingredients_country = json.load(f)

In [21]:
from collections import Counter
import numpy as np

def fit_tfidf(country_to_ingredients, lowercase=True, smooth_idf=True, sublinear_tf=False):
    # Build vocab
    vocab = set()
    docs = []
    for ingr_list in country_to_ingredients.values():
        doc = [x.lower() for x in ingr_list] if lowercase else list(ingr_list)
        docs.append(doc)
        vocab.update(doc)

    ingredient_to_id = {ing: i for i, ing in enumerate(sorted(vocab))}
    V = len(ingredient_to_id)
    N = len(docs)

    # Document frequency
    df = np.zeros(V, dtype=float)
    for doc in docs:
        for ing in set(doc):
            df[ingredient_to_id[ing]] += 1.0

    # IDF
    # classic smooth: idf = log((N+1)/(df+1)) + 1
    if smooth_idf:
        idf = np.log((N + 1.0) / (df + 1.0)) + 1.0
    else:
        idf = np.log(N / np.maximum(df, 1.0))

    # Build TF-IDF matrix for the training countries
    countries = list(country_to_ingredients.keys())
    X = np.zeros((N, V), dtype=float)

    for r, doc in enumerate(docs):
        tf = Counter(doc)
        for ing, c in tf.items():
            j = ingredient_to_id[ing]
            val = 1.0 + np.log(c) if sublinear_tf else float(c)
            X[r, j] = val

    X *= idf  # apply idf
    return countries, X, ingredient_to_id, idf


def transform_tfidf(ingredients, ingredient_to_id, idf, lowercase=True, sublinear_tf=False):
    V = len(ingredient_to_id)
    v = np.zeros(V, dtype=float)

    ingredients = [x.lower() for x in ingredients] if lowercase else list(ingredients)
    tf = Counter(ingredients)

    for ing, c in tf.items():
        j = ingredient_to_id.get(ing)
        if j is None:
            continue  # OOV ingredient -> ignore
        val = 1.0 + np.log(c) if sublinear_tf else float(c)
        v[j] = val

    v *= idf
    return v

def cosine_vec_to_matrix(v, X, eps=1e-12):
    vnorm = np.linalg.norm(v) + eps
    Xnorm = np.linalg.norm(X, axis=1) + eps
    return (X @ v) / (Xnorm * vnorm)

def cosine_matrix(X, eps=1e-12):
    # pairwise cosine sim for all countries
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    Xn = X / (norms + eps)
    return Xn @ Xn.T

In [22]:
countries, X, ingredient_to_id, idf = fit_tfidf(dict_ingredients_country, sublinear_tf=True)

In [23]:
dict_countries = defaultdict(int)
## Most similar countries in DB
S = cosine_matrix(X) # S[i,j] is cosine similarity between countries[i] and countries[j]

for i in range(len(countries)):
    country_vector = S[i].copy()
    country_vector[i] = -1 ## Just to make it so we don't select country[i] everytime since sim = 1
    best_matching_index = np.argmax(country_vector)
    best_matching_country = countries[best_matching_index]
    dict_countries[best_matching_country] += 1
print(dict_countries)


defaultdict(<class 'int'>, {'mongolia': 3, 'france': 7, 'greece': 3, 'chile': 1, 'japan': 2, 'angola': 1, 'netherlands': 2, 'canada': 2, 'mexico': 1, 'russia': 2, 'israel': 1, 'malaysia': 4, 'lebanon': 4, 'estonia': 4, 'hungary': 13, 'iraq': 1, 'dominican republic': 1, 'indonesia': 3, 'united states': 2, 'india': 6, 'singapore': 1, 'vietnam': 1, 'ecuador': 1, 'thailand': 1, 'cuba': 2, 'tunisia': 1, 'turkiye': 5, 'paraguay': 2, 'uzbekistan': 2, 'switzerland': 1, 'liechtenstein': 1, 'panama': 1, 'albania': 1, 'ukraine': 2, 'bolivia': 2, 'egypt': 2, 'moldova': 1, 'azerbaijan': 1, 'romania': 2, 'qatar': 2, 'norway': 3, 'mauritius': 1, 'tonga': 1, 'guam': 3, 'mozambique': 1, 'oman': 1, 'haiti': 1, 'china': 1, 'ghana': 1, 'lithuania': 2, 'belarus': 1, 'mauritania': 1, 'uruguay': 1, 'luxembourg': 2, 'trinidad and tobago': 1, 'belize': 1, 'spain': 1, 'pakistan': 1, 'chad': 1, 'brunei': 2, 'bulgaria': 1, 'antigua and barbuda': 1, 'afghanistan': 1, 'liberia': 1, 'yemen': 1, 'bahamas': 1, 'guerns

In [49]:
top_k = 10
most_common = sorted(dict_countries.items(), key=lambda x: x[1], reverse=True)[:top_k]
print(f"Most common countries:", most_common)

Most common countries: [('hungary', 13), ('france', 7), ('india', 6), ('turkiye', 5), ('malaysia', 4), ('lebanon', 4), ('estonia', 4), ('mongolia', 3), ('greece', 3), ('indonesia', 3)]


In [24]:
same_country = 0
same_country_2 = 0
same_ = 0
total = 0
total_tot = 0
list_countries_filter = list(dict_ingredients_country.keys())
countries, X, ingredient_to_id, idf = fit_tfidf(dict_ingredients_country, sublinear_tf=True)
dict_countries_mismatch = defaultdict(int)

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country'].lower()
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        same_novelty_dict = recipe_dict['LLM_gen']['same_country_novelty']
        same_key_list = list(same_novelty_dict.keys())
        for key in same_key_list:
            ingredient_list = same_novelty_dict[key]['Ingredients']
            ingredient_list = clean_ingredients_spacy(ingredient_list, list_countries_filter)
            v_new = transform_tfidf(ingredient_list, ingredient_to_id, idf, sublinear_tf=True)
            sims = cosine_vec_to_matrix(v_new, X)
            best_match, _ = sorted(zip(countries, sims), key=lambda x: -x[1])[0]

            if best_match == country_orig:
                same_country += 1
                same_country_2 += 1
                same_ += 1
            else:
                dict_countries_mismatch[best_match] += 1
            total += 1
            total_tot += 1

    varia_novelty_dict = recipe_dict['LLM_gen']['variation_novelty']
    varia_country_keys = list(varia_novelty_dict.keys())

    for varia_country in varia_country_keys:
        country_v = varia_country.split("_", 1)[1]
        key_list = list(varia_novelty_dict[varia_country].keys())
        for key in key_list:
            title = varia_novelty_dict[varia_country][key]['Title']
            ingredient_list = varia_novelty_dict[varia_country][key]['Ingredients']
            ingredient_list = clean_ingredients_spacy(ingredient_list, list_countries_filter)
            v_new = transform_tfidf(ingredient_list, ingredient_to_id, idf, sublinear_tf=True)
            sims = cosine_vec_to_matrix(v_new, X)
            best_match, _ = sorted(zip(countries, sims), key=lambda x: -x[1])[0]

            if best_match == country_orig:
                same_country_2 += 1
            elif best_match == country_v:
                same_ += 1
            else:
                dict_countries_mismatch[best_match] += 1
            total_tot += 1

  0%|          | 0/496 [00:00<?, ?it/s]

100%|██████████| 496/496 [37:58<00:00,  4.59s/it]


In [25]:
print('Proportion of recipes where ingredients match their origin country : ', same_country/total)
print('Proportion of recipes where ingredients match their origin country when varia: ', same_country_2/total_tot)
print('Proportion of inredients from varia match country varia : ', same_/(total_tot-total))

Proportion of recipes where ingredients match their origin country :  0.057305858310626706
Proportion of recipes where ingredients match their origin country when varia:  0.03952535354658773
Proportion of inredients from varia match country varia :  0.03243202115214518


In [52]:
top_k = 10
most_common = sorted(dict_countries_mismatch.items(), key=lambda x: x[1], reverse=True)[:top_k]
print(f"Most common countries:", most_common)

Most common countries: [('china', 28194), ('belarus', 11537), ('chad', 8285), ('liberia', 7816), ('guyana', 6693), ('finland', 6012), ('jamaica', 5209), ('qatar', 5015), ('dominican republic', 4910), ('estonia', 4602)]


In [30]:
def clean_ingr(ingredient, authorized_pos=['NOUN']):

    doc = nlp(ingredient)
    token_list = [token.text for token in doc if token.pos_ in authorized_pos]
    return token_list

In [ ]:
## Do for Humans too here
same_country = 0
same_country_2 = 0
same_country_v = 0
total = 0
list_countries_filter = list(dict_ingredients_country.keys())
countries, X, ingredient_to_id, idf = fit_tfidf(dict_ingredients_country, sublinear_tf=True)
dict_countries_mismatch = defaultdict(int)
total_tot = 0

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country'].lower()
    train_recipe_ids = list(recipe_dict['Train_Variations'].keys())
    for i in range(len(train_recipe_ids)):
        recipe = recipe_dict['Train_Variations'][train_recipe_ids[i]]
        country_t = recipe['country']
        ingredient_train = recipe['ingredient_list']
        human_ingredient_list = []
        for ingredient in ingredient_train:
            human_ingredient_list.extend(clean_ingr(ingredient))
        human_ingredient_list = list(set(human_ingredient_list))

        v_new = transform_tfidf(human_ingredient_list, ingredient_to_id, idf, sublinear_tf=True)
        sims = cosine_vec_to_matrix(v_new, X)
        best_match, _ = sorted(zip(countries, sims), key=lambda x: -x[1])[0]
        if country_t == country_orig:
            total += 1
        if best_match == country_orig:
            if country_t == country_orig:
                same_country_2 += 1
            same_country += 1
            total_tot += 1
        elif best_match == country_t:
            same_country_v += 1
            total_tot += 1
        else:
            dict_countries_mismatch[best_match] += 1
            total_tot += 1

    valid_recipe_ids = list(recipe_dict['Valid_Variations'].keys())
    for i in range(len(valid_recipe_ids)):
        recipe = recipe_dict['Valid_Variations'][valid_recipe_ids[i]]
        country_t = recipe['country']
        ingredient_valid = recipe['ingredient_list']
        
        human_ingredient_list = []
        for ingredient in ingredient_valid:
            human_ingredient_list.extend(clean_ingr(ingredient))
        human_ingredient_list = list(set(human_ingredient_list))
        v_new = transform_tfidf(human_ingredient_list, ingredient_to_id, idf, sublinear_tf=True)
        sims = cosine_vec_to_matrix(v_new, X)
        best_match, _ = sorted(zip(countries, sims), key=lambda x: -x[1])[0]

        if country_t == country_orig:
            total += 1
        if best_match == country_orig:
            if country_t == country_orig:
                same_country_2 += 1
            same_country += 1
            total_tot += 1
        elif best_match == country_t:
            same_country_v += 1
            total_tot += 1
        else:
            dict_countries_mismatch[best_match] += 1
            total_tot += 1


    test_recipe_ids = list(recipe_dict['Test_Variations'].keys())
    for i in range(len(test_recipe_ids)):
        recipe = recipe_dict['Test_Variations'][test_recipe_ids[i]]
        country_t = recipe['country']
        ingredient_test = recipe['ingredient_list']
        
        human_ingredient_list = []
        for ingredient in ingredient_test:
            human_ingredient_list.extend(clean_ingr(ingredient))
        human_ingredient_list = list(set(human_ingredient_list))

        v_new = transform_tfidf(human_ingredient_list, ingredient_to_id, idf, sublinear_tf=True)
        sims = cosine_vec_to_matrix(v_new, X)
        best_match, _ = sorted(zip(countries, sims), key=lambda x: -x[1])[0]

        if country_t == country_orig:
            total += 1
        if best_match == country_orig:
            if country_t == country_orig:
                same_country_2 += 1
            same_country += 1
            total_tot += 1
        elif best_match == country_t:
            same_country_v += 1
            total_tot += 1
        else:
            dict_countries_mismatch[best_match] += 1
            total_tot += 1 

In [59]:
print('Proportion of recipes where ingredients match their origin country : ', same_country_2/total)
print('Proportion of recipes where ingredients match their origin country : ', same_country/total_tot)
print('Proportion of recipes where ingredients match their origin country : ', same_country_v/total_tot)

Proportion of recipes where ingredients match their origin country :  0.06141732283464567
Proportion of recipes where ingredients match their origin country :  0.057351168264538725
Proportion of recipes where ingredients match their origin country :  0.019633733279751996


In [60]:
top_k = 10
most_common = sorted(dict_countries_mismatch.items(), key=lambda x: x[1], reverse=True)[:top_k]
print(f"Most common countries:", most_common)

Most common countries: [('china', 13889), ('haiti', 998), ('peru', 775), ('mongolia', 416)]


# Most common ingredients in LLMs production

In [26]:
list_countries_filter = list(dict_ingredients_country.keys())
countries, X, ingredient_to_id, idf = fit_tfidf(dict_ingredients_country, sublinear_tf=True)
dict_ingredients = defaultdict(int)

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    country_orig = recipe_dict['Country'].lower()
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        same_novelty_dict = recipe_dict['LLM_gen']['same_country_novelty']
        same_key_list = list(same_novelty_dict.keys())
        for key in same_key_list:
            ingredient_list = same_novelty_dict[key]['Ingredients']
            ingredient_list = clean_ingredients_spacy(ingredient_list, list_countries_filter)
            for ingredient in ingredient_list:
                dict_ingredients[ingredient] += 1

    varia_novelty_dict = recipe_dict['LLM_gen']['variation_novelty']
    varia_country_keys = list(varia_novelty_dict.keys())

    for varia_country in varia_country_keys:
        country_v = varia_country.split("_", 1)[1]
        key_list = list(varia_novelty_dict[varia_country].keys())
        for key in key_list:
            title = varia_novelty_dict[varia_country][key]['Title']
            ingredient_list = varia_novelty_dict[varia_country][key]['Ingredients']
            ingredient_list = clean_ingredients_spacy(ingredient_list, list_countries_filter)
            for ingredient in ingredient_list:
                dict_ingredients[ingredient] += 1

  0%|          | 0/496 [00:00<?, ?it/s]

100%|██████████| 496/496 [36:46<00:00,  4.45s/it]


In [27]:
top_k = 20
most_common = sorted(dict_ingredients.items(), key=lambda x: x[1], reverse=True)[:top_k]
print(f"Most common countries:", most_common)

Most common countries: [('salt', 32383), ('oil', 30303), ('butter', 26741), ('sugar', 26307), ('powder', 25519), ('vanilla extract', 22283), ('pepper', 20359), ('onion', 20002), ('ground', 17875), ('parsley', 17723), ('ground cumin', 14213), ('lemon juice', 14073), ('cheese', 12621), ('ground cinnamon', 12488), ('salt taste', 11898), ('', 10996), ('oregano', 10430), ('water', 10154), ('egg', 10055), ('milk', 9478)]
